# THIS IS DEPRICATED

# Imports

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
use_old = False # model
use_old_data = False

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import keras
from keras.layers import Input, Dense, Lambda, Conv2D, Flatten, Conv2DTranspose, Reshape, Activation
from keras.models import Model, Sequential
from keras import backend as K
from keras import objectives
from keras.datasets import mnist
from keras.activations import softmax
from keras.objectives import binary_crossentropy as bce
from keras.objectives import mean_squared_error as mse

%pylab inline
import os
from skimage.transform import resize


In [ ]:
# import sys
# sys.path.insert(0, "/media/toke/Armory/tank/git/World-Models-TensorFlow/stateless_agent")
from train_Gumbel_VAE import Network, show_pred

In [ ]:
data_dir = "../data"

if use_old_data:
    data1 = "small_obs_data_car_racing_1.npy"

    np_dat = np.load(os.path.join(data_dir, data1))
    x_train = np_dat[:8000]
    x_test = np_dat[8000:]
    print(x_train.shape, x_test.shape)
    print(np.min(x_train), np.max(x_train))
    print(x_train.dtype)

    plt.imshow(x_train[70])
else:

    data1 = "obs_data_VAE_0.npy"
    print(data1)
    np_dat = np.load(os.path.join(data_dir, data1))

    data1 = "obs_data_VAE_1.npy"
    print(data1)
    data = np.load(os.path.join(data_dir, data1))
    np_dat = np.concatenate([np_dat, data])

    data1 = "obs_data_VAE_2.npy"
    print(data1)
    data = np.load(os.path.join(data_dir, data1))
    np_dat = np.concatenate([np_dat, data])

    data1 = "obs_data_VAE_3.npy"
    print(data1)
    data = np.load(os.path.join(data_dir, data1))
    np_dat = np.concatenate([np_dat, data])


    np_dat = resize(np_dat, (len(np_dat), 64, 64, 3), anti_aliasing=True).astype(np.float32)
    x_train = np_dat[:8000]
    x_test = np_dat[8000:]
    print(x_train.shape, x_test.shape)
    print(np.min(x_train), np.max(x_train))
    print(x_train.dtype)

    plt.imshow(x_train[89])

# Model

In [ ]:
def sampling(logits_y):
    U = K.random_uniform(K.shape(logits_y), 0, 1)
    y = logits_y - K.log(-K.log(U + 1e-20) + 1e-20) # logits + gumbel noise
    y = softmax(K.reshape(y, (-1, N, M)) / tau)
#     y = softmax(K.reshape(y, (N, M)) / tau,axis=-1)
    y = K.reshape(y, (-1, N*M))
    return y


# def gumbel_loss(x, x_hat):
#     q_y = K.reshape(logits_y, (-1, N, M))
#     q_y = softmax(q_y)
#     log_q_y = K.log(q_y + 1e-20)
#     kl_tmp = q_y * (log_q_y - K.log(1.0/M))
#     KL = K.sum(kl_tmp, axis=(1, 2))
# #     elbo = data_dim * bce(x, x_hat) - KL 
#     elbo = DATA_DIM * bce(x, x_hat) - KL
#     return elbo

# def gumbel_loss(x, x_hat):
#     print(K.shape(logits_y))
#     q_y = K.reshape(logits_y, (-1, N, M))
#     print(K.shape(logits_y))
#     q_y = softmax(q_y)
#     log_q_y = K.log(q_y + 1e-20)
#     kl_tmp = q_y * (log_q_y - K.log(1.0/M))
#     KL = K.sum(kl_tmp, axis=(1, 2))
# #     elbo = data_dim * bce(x, x_hat) - KL 
#     elbo = DATA_DIM * bce(x, x_hat) - KL
#     return elbo

def gumbel_loss(x, x_hat):
    q_y = K.reshape(logits_y, (-1, N, M))
    q_y = softmax(q_y)
    log_q_y = K.log(q_y + 1e-20)
    kl_tmp = q_y * (log_q_y - K.log(1.0/M))
    KL = K.sum(kl_tmp, axis=(1, 2))
    x = K.reshape(x, (1,-1))
    x_hat = K.reshape(x_hat, (1,-1))
#     elbo = DATA_DIM * bce(x, x_hat) - KL
    elbo = DATA_DIM * mse(x, x_hat) - KL
    return elbo


In [ ]:
def make_plots():
    argmax_y = K.max(K.reshape(logits_y, (-1, N, M)), axis=-1, keepdims=True)
    argmax_y = K.equal(K.reshape(logits_y, (-1, N, M)), argmax_y)
    # encoder = K.function([x], [argmax_y, x_hat])
    encoder = K.function([x], [argmax_y, decoded_x])

    # test_idx, try: from 160-180
    test_idx = np.random.randint(len(x_test))
    _batch = len(x_test)
    code, x_hat_test = encoder([x_test[:_batch]])

    _x_hat, _logits, _z = vae_test.predict(x_test[:_batch])

    subplot(331)
    title("Input image")
    # imshow(x_test[test_idx].reshape(28, 28), cmap='gray'), axis('off')
    imshow(x_test[test_idx].reshape(64, 64, 3), cmap='gray'), axis('off')

    subplot(334)
    title("true")
    imshow(_z[test_idx].reshape(N, M), cmap='gray'), axis('off')

    subplot(335)
    title("decoder")
    img_true = generator.predict(np.reshape(_z[test_idx], (1,-1)) )
    # imshow(img_true.reshape(28, 28), cmap='gray'), axis('off')
    imshow(img_true.reshape(64, 64, 3), cmap='gray'), axis('off')

    subplot(336)
    title("full network")
    # imshow(x_hat_test[test_idx].reshape(28, 28), cmap='gray'), axis('off')
    imshow(x_hat_test[test_idx].reshape(64, 64, 3), cmap='gray'), axis('off')


    # imshow(x_hat_test[test_idx].reshape(28, 28), cmap='gray'), axis('off')
    # imshow(x_hat_test[test_idx].reshape(64, 64, 3), cmap='gray'), axis('off')

    subplot(337)
    title("binary")
    imshow(code[test_idx].reshape(N, M), cmap='gray'), axis('off')

    subplot(338)
    title("binary x_hat")
    img_binary = generator.predict(np.reshape(code[test_idx], (1,-1)) )
    # imshow(img_binary.reshape(28, 28), cmap='gray'), axis('off')
    imshow(x_hat_test[test_idx].reshape(64, 64, 3), cmap='gray'), axis('off')

    plt.show()


# Model 3

In [ ]:
batch_size = 64
INPUT_DIM = (64,64,3)
DATA_DIM = np.prod(INPUT_DIM) 
N = 32  # Number var
M = 8  #  Number values per var
nb_epoch = 500 #100

#tau = K.variable(5.0, name="temperature")
anneal_rate = np.log(2)/20
print(anneal_rate)
min_temperature = 0.01

In [ ]:
tau = K.variable(5.0, name="temperature")

if use_old:
    ACTIV = "relu"

    # x = Input(batch_shape=(batch_size, data_dim ))
    x = Input(shape=INPUT_DIM)
    # h = Dense(256, activation='relu')(Dense(512, activation='relu')(x))
    h = Conv2D(filters=32, kernel_size=4, strides=2, padding='same', activation=ACTIV)(x)
    h = Conv2D(filters=64, kernel_size=4, strides=2, padding='same', activation=ACTIV)(h)
    # h = Conv2D(filters=64, kernel_size=3, strides=2, padding='valid', activation=ACTIV)(h)
    h = Flatten()(h)

    logits_y = Dense(M*N, activation=None)(h)

    # z = Lambda(sampling, output_shape=(M*N,))(logits_y)
    z_lay = Lambda(sampling, output_shape=(M*N,))
    z = z_lay(logits_y)
    z = Activation(None)(z)

    # z = Reshape((1,1,N*M))(z)
    # z = Conv2DTranspose(filters=32, kernel_size=6, strides=4, padding="same")(z)
    # z = Conv2DTranspose(filters=32, kernel_size=3, strides=4, padding="same")(z)
    # z = Conv2DTranspose(filters=16, kernel_size=3, strides=2, padding="same")(z)
    # x_hat = Conv2DTranspose(filters=3, kernel_size=3, strides=2, padding="same")(z)

    z_1 = Reshape((1,1,N*M))
    z_2 = Conv2DTranspose(filters=32, kernel_size=4, strides=2, padding="same", activation=ACTIV)
    z_3 = Conv2DTranspose(filters=32, kernel_size=4, strides=2, padding="same", activation=ACTIV)
    z_4 = Conv2DTranspose(filters=16, kernel_size=4, strides=2, padding="same", activation=ACTIV)
    z_5 = Conv2DTranspose(filters=16, kernel_size=4, strides=4, padding="same", activation=ACTIV)
    x_hat = Conv2DTranspose(filters=3, kernel_size=4, strides=2, padding="same", activation="sigmoid")

    decode_input = Input( shape=(M*N,) )
    decoder = x_hat(z_5(z_4(z_3(z_2(z_1( decode_input ))))))
    generator = Model(decode_input, decoder)
    
    decoded_x = x_hat(z_5(z_4(z_3(z_2(z_1( z ))))))
    vae = Model(x, decoded_x)
    vae.compile(optimizer='adam', loss=gumbel_loss)
    vae.summary()

    vae_test = Model(x, [decoded_x, logits_y, z])
    
else:
    vae = Network(N,M)
    vae.tau =  tau


# Train

In [ ]:
for e in range(nb_epoch):
    print(e, "/", nb_epoch, '- K:', K.get_value(tau))
    if use_old:
        K.set_value(tau, np.max([min_temperature, 5.0 * np.exp(- anneal_rate * e)]))
        train_hist = vae.fit(x_train, x_train, 
        shuffle=True, 
        epochs=1, 
        batch_size=batch_size,
        validation_data=(x_test, x_test))
    else:
        K.set_value(vae.tau, np.max([min_temperature, 5.0 * np.exp(- anneal_rate * e)]))
        train_hist = vae.model.fit(x_train, x_train, 
            shuffle=True, 
            epochs=1, 
            batch_size=batch_size,
            validation_data=(x_test, x_test))

    if e % 2==0 or e == nb_epoch-1:
        fig = plt.figure(1, figsize=(10,5))
        if use_old:
            make_plots()
       
        else:
            print("Validation score", vae.model.evaluate(x_test, x_test))

            logits, gumbel, pred = vae.tester.predict(x_test)
            n = np.random.randint(len(x_test))
            subplot(221)
            plt.imshow(x_test[n])
            title = "e{} n{}".format(e,n)
            subplot(222)
            plt.imshow(pred[n])
            plt.tight_layout()

            subplot(223)
            plt.imshow(np.reshape(gumbel[n], (N,M)).T)
            # plt.colorbar()
            subplot(224)
            plt.imshow(np.reshape(logits[n], (N,M)).T)
            plt.colorbar()
            plt.tight_layout()
            plt.show()
#             show_pred(title, x_test, pred)


In [ ]:
gumbel.shape

## Save model weights

In [ ]:

# vae.save_weights('model3_vae_weights.h5')

# Visualize results

In [ ]:
argmax_y = K.max(K.reshape(logits_y, (-1, N, M)), axis=-1, keepdims=True)
argmax_y = K.equal(K.reshape(logits_y, (-1, N, M)), argmax_y)
# encoder = K.function([x], [argmax_y, x_hat])
encoder = K.function([x], [argmax_y, decoded_x])


# Create sample 1.0

In [ ]:
%matplotlib inline
from ipywidgets import interactive
from ipywidgets import widgets
from IPython.display import display

In [ ]:
# N = 10
# M = 5

img_code = np.zeros( (N,M) )

def f(*args,**kwargs):
#     print(kwargs)
    for i, key in enumerate(kwargs):
#         print( kwargs[key] )
        zero_row = np.zeros((1,M))
        zero_row[0, kwargs[key] ] = 1
        img_code[i] = zero_row
    
    plt.subplot(121)
    plt.imshow(img_code, cmap="gray")
    
    plt.subplot(122)
    img_binary = generator.predict(np.reshape(img_code, (1,-1)) )
#     plt.imshow(img_binary.reshape(28, 28), cmap='gray')
    plt.imshow(img_binary.reshape(64, 64, 3), cmap='gray')
    plt.axis('off')


kw_dict = {}
for n in range(N):
    kw_dict["z"+str(n)] = widgets.IntSlider(description="z"+str(n), min=0, max=(M-1), value=int(M/2), continuous_update=False)

interactive_plot = interactive(f, **kw_dict)
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot

# Create sample 2.0

In [ ]:
sample = np.zeros( (N,M) )

for i in range(N):
    sample[i, np.random.randint(0,M)] = 1.0

# print(sample)
# imshow(sample, cmap='gray'), axis('off')
imshow(code[test_idx].reshape(N, M), cmap='gray'), axis('off')

In [ ]:
# def softmaxx(x):
#     """Compute softmax values for each sets of scores in x."""
#     return np.exp(x) / np.sum(np.exp(x), axis=0)

def softmaxx(x):
    """Compute softmax values for each sets of scores in x."""
    arr = []
    for row in x:
        e_x = np.exp(row - np.max(row))
        e_x = e_x / e_x.sum()
        arr.append(e_x)
#         print(e_x)
    return np.asarray(arr)


def sampl(logits_y, tau):
#     U = K.random_uniform(K.shape(logits_y), 0, 1)
#     print("LOGITS_Y")
#     print(logits_y)
    U = np.random.uniform(0, 1, logits_y.shape)
#     print("U !!!")
#     print(U)
    y = logits_y - np.log(-np.log(U + 1e-20) + 1e-20) # logits + gumbel noise
#     print("y: logits minus gumbel")
#     print(y)
    y = softmaxx(np.reshape(y, (-1, N, M)) / tau)
#     print("y after softmax")
#     print(y)
    y = np.reshape(y, (-1, N*M))
#     print("y after reshape")
#     print(y)
    return y



In [ ]:
def sample_gumbel(shape, eps=1e-20): 
  """Sample from Gumbel(0, 1)"""
  U = tf.random_uniform(shape,minval=0,maxval=1)
  return -tf.log(-tf.log(U + eps) + eps)

def gumbel_softmax_sample(logits, temperature): 
  """ Draw a sample from the Gumbel-Softmax distribution"""
  y = logits + sample_gumbel(tf.shape(logits))
  return tf.nn.softmax( y / temperature)


In [ ]:
a = sampl( np.random.rand(M*N), 0.1 )
# b = sampl( np.random.rand(M*N)*100, 0.1 )
# c = sampl( np.zeros((1,M*N)), 0.1 )

# b = np.reshape(b, (N,M))
# b




In [ ]:
# code, x_hat_test = encoder([x_test[:1]])
rand_code = np.eye(M)[np.random.choice(M, N)]
rand_img = generator.predict(np.reshape(rand_code, [-1,N*M]))
sampl_code = bli
sampl_img =  generator.predict(np.reshape(sampl_code, [-1,N*M]))

In [ ]:
# img_reshaped = rand_img.reshape(28,28)
img_reshaped = sampl_img.reshape(28,28)
img_reshaped.shape
plt.imshow(img_reshaped, cmap="gray")